In [1]:
using Dates: Date, DateTime;
using Printf
using JuMP

In [2]:
PTDF_TRIMMER = 1e-6;

root_path = @__DIR__;
push!(LOAD_PATH, root_path);
cd(root_path);

include(joinpath(root_path, "scopf_utils.jl"))
include(joinpath(root_path, "AmplTxt.jl"));
include(joinpath(root_path, "SCOPF.jl"));
include(joinpath(root_path, "Workflow.jl"));

optimizer: Cbc.Optimizer


# 5-buses

In [3]:
base_data_name = "data/data_slack/5buses/base";

## base PTDF with bus 1 as reference

In [4]:
amplTxt = AmplTxt.read(base_data_name);
base_data_path = joinpath(root_path, base_data_name)
network = SCOPF.Network(amplTxt);

ref_bus = 1;
B = SCOPF.get_B(network, 1e-6);
Binv = SCOPF.get_B_inv(B, ref_bus);
PTDF = SCOPF.get_PTDF(network, Binv, ref_bus);

file_path_l =  joinpath(base_data_path, "ptdf.txt")
SCOPF.write_PTDF(file_path_l, network, ref_bus, PTDF, PTDF_TRIMMER)

n = 5
ref_bus is 1
n 5
m 6


In [5]:
network.bus_to_i

Dict{Int64, Int64} with 5 entries:
  5 => 5
  4 => 4
  2 => 2
  3 => 3
  1 => 1

In [6]:
#WARNING buses 2 and 3 are reversed
network.buses

Dict{Int64, Main.SCOPF.Bus} with 5 entries:
  5 => Bus(5, "5")
  4 => Bus(4, "4")
  2 => Bus(2, "3")
  3 => Bus(3, "2")
  1 => Bus(1, "1")

In [7]:
network.branches

Dict{Int64, Main.SCOPF.Branch} with 6 entries:
  5 => Branch(4, 5, 5.61437e-6, 5.61437e-5, "4_5")
  4 => Branch(1, 4, 5.74669e-6, 5.74669e-5, "1_4")
  6 => Branch(5, 1, 1.20983e-6, 1.20983e-5, "5_1")
  2 => Branch(3, 2, 2.04159e-6, 2.04159e-5, "2_3")
  3 => Branch(2, 4, 5.61437e-6, 5.61437e-5, "3_4")
  1 => Branch(1, 3, 5.31191e-6, 5.31191e-5, "1_2")

## generate multiple coeffs distribution for PTDFs calculation

In [8]:
nb_buses = size(PTDF)[2]

function gen_coeffs(nb_buses_p)
    dict_coeffs = Dict{String, Vector{Float64}}()

    dict_coeffs["uniform"] = ones(nb_buses_p)/nb_buses_p

    # generate coefficients each time giving a coeff of 1. to a selected bus
    for dominant_bus_l in range(1,nb_buses_p)
        coeffs_l = zeros(nb_buses_p)
        coeffs_l[dominant_bus_l] = 1.
        test_name = @sprintf("bus_%d", dominant_bus_l)
        dict_coeffs[test_name] = coeffs_l
    end
    
    return dict_coeffs
end

#dict_coeffs = gen_coeffs(nb_buses)
dict_coeffs = Dict( "uniform" => [0.2, 0.2, 0.2, 0.2, 0.2],
                    "bus4" => [0., 0., 0., 1., 0.],
                    )


SCOPFutils.pretty_print(dict_coeffs, 3)

   bus4 => [0.0, 0.0, 0.0, 1.0, 0.0]
   uniform => [0.2, 0.2, 0.2, 0.2, 0.2]


## launch PSCOPF with different PTDFs

In [9]:
ECH = DateTime("2015-01-01T11:00:00")
P_RES_MIN = 0; #use positive value
P_RES_MAX = 500;
@assert((P_RES_MIN>=0) & (P_RES_MAX>=0) & (P_RES_MIN<=P_RES_MAX))

TS = DateTime("2015-01-01T11:00:00")
SCENARIO = "S1"

#dict_flows[reserve][branch][testname] = flow 
dict_flows = Dict{Float64, Dict{String, Dict{String, Float64}}}()
dict_flows[P_RES_MAX] = Dict{String, Dict{String, Float64}}()
#get!(dict_flows, 105., Dict{String, Float64}())["az"] = 15.
dict_prod = Dict{Float64, Dict{String, Dict{String, Float64}}}()
dict_prod[P_RES_MAX] = Dict{String, Dict{String, Float64}}()

for (test_name_l, coeffs_l) in dict_coeffs
    usecase_name_l = @sprintf("%s_RES_%s_%s", test_name_l, P_RES_MIN, P_RES_MAX)
    usecase_path_l = joinpath(dirname(rstrip(base_data_path,'/')), usecase_name_l)
    mkdir(usecase_path_l)
    PTDF_distributed = SCOPF.distribute_slack(PTDF, 1, coeffs_l);
    
    println(usecase_path_l, " :", coeffs_l)
    file_path_l = joinpath(usecase_path_l, "pscopf_ptdf.txt")
    SCOPF.write_PTDF(file_path_l, network, ref_bus, PTDF_distributed, PTDF_TRIMMER)
    file_path_l = joinpath(usecase_path_l, "coeffs.txt")
    SCOPF.write_slack_distribution(file_path_l, network, coeffs_l)
    
    #read input from "base" folder but write into the "test" folder
    launcher_l = Workflow.Launcher(base_data_path, usecase_path_l);
    
    launcher_l.ptdf = Workflow.read_ptdf(usecase_path_l)
    
    model, v_lim, v_imp, v_res, v_flow = Workflow.sc_opf(launcher_l, ECH, -P_RES_MIN, P_RES_MAX);
    for ((branch_l, _, s), flow_var_l) in v_flow
        get!(dict_flows[P_RES_MAX], branch_l, Dict{String, Float64}());
        if s == SCENARIO
            dict_flows[P_RES_MAX][branch_l][test_name_l] = value(flow_var_l)
        end
    end
    
    #dict_prod[P_RES_MAX][unit_l][test_name_l]
    for (kind_l, dict_units) in Workflow.get_units_by_kind(launcher_l)
        for (unit_l, _) in dict_units
            get!(dict_prod[P_RES_MAX], unit_l, Dict{String, Float64}());
            if kind_l == Workflow.K_IMPOSABLE
                dict_prod[P_RES_MAX][unit_l][test_name_l] = value(v_imp.p_imposable[unit_l, TS, SCENARIO])
            elseif kind_l == Workflow.K_LIMITABLE
                dict_prod[P_RES_MAX][unit_l][test_name_l] = value(v_lim.p_enr[unit_l, TS, SCENARIO])
            end
        end
    end
end

dict_flows

/home/aziz/Projects/RTE/PSCOPF/pscopf/data/data_slack/5buses/bus4_RES_0_500 :[0.0, 0.0, 0.0, 1.0, 0.0]
NAMES : Set(["wind_3", "4", "1", "2", "wind_4", "5", "fuel_3", "wind_1", "fuel_4", "fuel_1", "wind_5", "wind_2", "fuel_5", "gas_1", "3"])
BUSES : Set(["4", "1", "5", "2", "3"])
units_by_bus : Dict("Imposable" => Dict("4" => ["fuel_4"], "1" => ["gas_1", "fuel_1"], "5" => ["fuel_5"], "2" => [], "3" => ["fuel_3"]), "Limitable" => Dict("4" => ["wind_4"], "1" => ["wind_1"], "5" => ["wind_5"], "2" => ["wind_2"], "3" => ["wind_3"]))
Number of scenario 1
Number of time step is 1
[DateTime("2015-01-01T11:00:00")]
eod_slack is Dict((DateTime("2015-01-01T11:00:00"), "S1") => -10.0)
fuel_4 2015-01-01T11:00:00 is at 0.000000

fuel_5 2015-01-01T11:00:00 is at 0.000000

gas_1 2015-01-01T11:00:00 is at 0.000000

fuel_1 2015-01-01T11:00:00 is at 0.000000

fuel_3 2015-01-01T11:00:00 is at 0.000000

ptdf_by_branch : Dict("3_4" => Dict("1" => 0.1939165, "5" => 0.159538, "2" => 0.5241051, "3" => 0.6510105

ZeroHalf was tried 1 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seend of optim.
OPTIMAL
525.4331190940467
NO_IMPOSABLE   : false
NO_LIMITABLE   : false
NO_LIMITATION  : false
BUSES          : Set(["4", "1", "5", "2", "3"])
NAMES          : Set(["wind_3", "4", "1", "2", "wind_4", "5", "fuel_3", "wind_1", "fuel_4", "fuel_1", "wind_5", "wind_2", "fuel_5", "gas_1", "3"])
TS             : [DateTime("2015-01-01T11:00:00")]
S              : ["S1"]
                           gen[ts,s] b_enr      p_enr  p_lim_enr         p0
    wind_1[2015-01-01T11:00:00,S1]    1     70.907     70.907    100.000
    wind_5[2015-01-01T11:00:00,S1]    1     76.556     76.556    100.000
v_res.p_res_pos
(DateTime("2015-01-01T11:00:00"), "S1") => p_res_pos[2015-01-01T11:00:00,S1] = 62.537038205584274
v_res.p_res_neg
v_res.v_flow
("3_4", DateTime("2015-01-01T11:00:00"), "S1") => p_flow[3_4, 2015-01-01T11:00:00,S1] = 97.49259165888319
("1_2", DateTime("2015-01-01T11:00:00"), "S1

Dict{Float64, Dict{String, Dict{String, Float64}}} with 1 entry:
  500.0 => Dict("3_4"=>Dict("bus4"=>85.2769, "uniform"=>97.4926), "1_4"=>Dict("…

In [10]:
SCOPFutils.pretty_print(dict_flows)

 500.0 => 
           "3_4" => 
                     bus4 => 85.27691308153321
                     uniform => 97.49259165888319
           "1_4" => 
                     bus4 => 100.0
                     uniform => 99.99999999999999
           "4_5" => 
                     bus4 => -100.00000000000001
                     uniform => -100.0
           "1_2" => 
                     bus4 => -14.723086918466791
                     uniform => -27.5222143233505
           "5_1" => 
                     bus4 => -10.93703218713828
                     uniform => -10.936962371655019
           "2_3" => 
                     bus4 => 85.27691308153321
                     uniform => 84.98518243017057


In [11]:
SCOPFutils.pretty_print(dict_prod)

 500.0 => 
           "wind_1" => 
                        bus4 => 96.21396489006798
                        uniform => 70.90732760718767
           "fuel_4" => 
                        bus4 => 0.0
                        uniform => 0.0
           "fuel_5" => 
                        bus4 => 0.0
                        uniform => 0.0
           "wind_2" => 
                        bus4 => 100.0
                        uniform => 100.0
           "gas_1" => 
                       bus4 => 0.0
                       uniform => 0.0
           "fuel_1" => 
                        bus4 => 0.0
                        uniform => 0.0
           "fuel_3" => 
                        bus4 => 0.0
                        uniform => 0.0
           "wind_4" => 
                        bus4 => 100.0
                        uniform => 100.0
           "wind_5" => 
                        bus4 => 89.06296781286171
                        uniform => 76.55563418722811
           "wind_3" => 
             